In [11]:
import torch
import torch.nn as nn
from torch.nn.parallel import DataParallel

import os
import re
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"    # Free GPU
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    cache_dir="/data5/home/sahilm/NLP_Project/Llama_2_7b_chat_hf"
    )

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", 
                                          cache_dir="/data5/home/sahilm/NLP_Project/Llama_2_7b_chat_hf"
                                         )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
if torch.cuda.device_count() > 1:
    model = DataParallel(model)
    print("Moj")
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Moj


DataParallel(
  (module): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm): LlamaRMS

In [5]:
model_without_parallel = model.module
def get_llama2_reponse(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature= 0.00001)
    outputs = model_without_parallel.generate(**inputs, temperature = 1e-4)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [6]:
def find_sentence_between_last_triple_backticks(text):
    matches = re.findall(r'```([^`]*)```', text)
    if matches:
        return matches[-1]
    else:
        return None

In [9]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

def calculate_bleu_score(reference, candidate):
    # Tokenize reference and candidate strings
    reference_tokens = word_tokenize(reference.lower())
    candidate_tokens = word_tokenize(candidate.lower())

    # Calculate BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)
    
    return bleu_score

# Example usage:
reference = "this is a reference sent ence"
candidate = "this is a reference sent ence"
score = calculate_bleu_score(reference, candidate)
print("BLEU Score:", score)


BLEU Score: 1.0


In [ ]:
# Open both files
with open("/data5/home/sahilm/NLP_Project/Dataset/IndicNECorp1.0/English-Mizo/parallel/en-lus-test-en.txt", "r") as file1, open("/data5/home/sahilm/NLP_Project/Dataset/IndicNECorp1.0/English-Mizo/parallel/en-lus-test-lus.txt", "r") as file2:
    # Iterate over lines from both files simultaneously
    cnt = 0
    for line1, line2 in zip(file1, file2):
        cnt+=1
        # Process the lines (e.g., print them)
        # print("Line from file1:", line1.strip())
        # print("Line from file2:", line2.strip())
        text = line1.strip()[:20]
        template = f"""Translate the following text to Hindi:\nText: {text}\nOutput: """    
        # print(template)
        # print(text)
        output = get_llama2_reponse(template)
        # print(output)
        # output1 = output.split("```")

        sentences_after_output = re.findall(r'Output:(.*)', output)
        print(sentences_after_output[0].strip())

        if cnt==10:
            break
        # print(find_sentence_between_last_triple_backticks(output))

